# 生成销售数据

**销售人员表 (Salesperson):**

*   id (主键，自增长)
*   name (销售人员姓名)
*   region (所属大区)

**商品信息表 (Product):**

*   id (主键，自增长)
*   name (商品名称)
*   price (商品价格)

**客户资料表 (Customer):**

*   id (主键，自增长)
*   name (客户姓名)
*   region (所属大区)

**销售数据表 (SalesData):**

*   id (主键，自增长)
*   salesperson\_id (外键，关联销售人员表的id)
*   product\_id (外键，关联商品信息表的id)
*   customer\_id (外键，关联客户资料表的id)
*   amount (销售金额)
*   sales\_date (销售日期)


In [ ]:
# %pip install pymysql cryptography faker

In [80]:
import random
import faker
import pymysql

## 连接数据库

![erd](./img/erd.png)


In [150]:
# 建立数据库连接
cnx = pymysql.connect(
    host='localhost',
    user='root',
    password='04m27d',
    database='sale',
    charset='utf8mb4'
)
cursor = cnx.cursor()

## 创建数据库表


In [82]:
# 创建Region表
create_table = """
CREATE TABLE Region (
  id INT AUTO_INCREMENT PRIMARY KEY,
  region_name VARCHAR(255)
)
"""
cursor.execute(create_table)


# 创建销售人员表
salesperson_table = """
CREATE TABLE Salesperson (
  id INT AUTO_INCREMENT PRIMARY KEY,
  name VARCHAR(255),
  gender CHAR(1),
  birthday DATE,
  address VARCHAR(255),
  id_card VARCHAR(255),
  phone VARCHAR(255),
  performance_score DECIMAL(5, 2),
  performance_bonus DECIMAL(10, 2),
  position VARCHAR(255),
  join_date DATE,
  region_id INT,
  FOREIGN KEY (region_id) REFERENCES Region(id)
)
"""
cursor.execute(salesperson_table)

# 创建商品信息表
product_table = """
CREATE TABLE Product (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100),
    price DECIMAL(10, 2)
)
"""
cursor.execute(product_table)

# 创建客户资料表
customer_table = """
CREATE TABLE Customer (
  id INT AUTO_INCREMENT PRIMARY KEY,
  name VARCHAR(255),
  region_id INT,
  FOREIGN KEY (region_id) REFERENCES Region(id)
)
"""
cursor.execute(customer_table)

# 创建销售数据表
sales_data_table = """
CREATE TABLE SalesData (
    id INT AUTO_INCREMENT PRIMARY KEY,
    salesperson_id INT,
    product_id INT,
    customer_id INT,
    region_id INT,
    amount INT,
    total DECIMAL(10, 2),
    sales_date DATE,
    FOREIGN KEY (salesperson_id) REFERENCES Salesperson(id),
    FOREIGN KEY (product_id) REFERENCES Product(id),
    FOREIGN KEY (customer_id) REFERENCES Customer(id),
    FOREIGN KEY (region_id) REFERENCES Region(id)
)
"""
cursor.execute(sales_data_table)

0

## 生成大区信息


In [83]:
# 区域信息
regions = ['东北', '华北', '华东', '华南', '西南']

# 插入区域数据
insert_regions = "INSERT INTO Region (region_name) VALUES (%s)"
region_data = [(region,) for region in regions]
cursor.executemany(insert_regions, region_data)

# 提交事务
cnx.commit()

## 读取大区信息


In [118]:
query_regions = "SELECT id FROM Region"
cursor.execute(query_regions)
regions = [row[0] for row in cursor.fetchall()]

regions

[1, 2, 3, 4, 5]

## 生成销售人员信息


In [147]:
# 模拟生成销售人员数据
from faker import Faker


# 生成模拟数据
fake = Faker('zh_CN')

salesperson_data = []
for region in regions:
    # 生成每个大区的销售人员数量
    salesperson_count = random.randint(70, 150)

    for _ in range(salesperson_count):
        profile = fake.profile()
        name = profile['name']
        gender = profile['sex']
        birthday = fake.date_of_birth(minimum_age=25, maximum_age=50)
        address = profile['address']
        phone = fake.phone_number()
        performance_score = random.uniform(60, 100)
        performance_bonus = random.uniform(1000, 5000)
        position = position = random.choice(['销售经理', '销售主管', '销售代表'])
        join_date = fake.date_between(start_date='-5y', end_date='today')
        id_card = profile['ssn']

        salesperson_data.append((name, gender, birthday, address, phone, performance_score,
                                performance_bonus, position, join_date, id_card, region))

# 插入销售人员数据
insert_salesperson = "INSERT INTO Salesperson (name, gender, birthday, address, phone, performance_score, performance_bonus, position, join_date, id_card, region_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
cursor.executemany(insert_salesperson, salesperson_data)

# 提交事务
cnx.commit()

## 生成商品数据


### 获取商品名称

#### 获取淘宝商品信息

##### 获取指定商品列表的url


In [87]:
# 导入requests包
import requests

# `q`即为查询的商品的名称, `x`为查询的页数


def Geturls(q, x):
    url = "https://s.taobao.com/search?q=" + q + "&imgfile=&commend=all&ssid=s5-e&search_type=item&sourceId=tb.index&spm" \
                                                 "=a21bo.2017.201856-taobao-item.1&ie=utf8&initiative_id=tbindexz_20170306"
    urls = []
    urls.append(url)
    if x == 1:
        return urls
    for i in range(1, x):
        url = "https://s.taobao.com/search?q=" + q + "&commend=all&ssid=s5-e&search_type=item" \
              "&sourceId=tb.index&spm=a21bo.2017.201856-taobao-item.1&ie=utf8&initiative_id=tbindexz_20170306" \
              "&bcoffset=3&ntoffset=3&p4ppushleft=1%2C48&s=" + str(
                  i * 44)
        urls.append(url)
    return urls

##### 获取商品列表HTML页面


In [88]:
def GetHtml(url, headers):
  # 这里的请求get请求
    r = requests.get(url, headers=headers)
    r.raise_for_status()
    r.encoding = r.apparent_encoding
    return r

##### 解析商品列表HTML页面


In [89]:
import json
import re
import time


def get_goods_info(goods_str):
    """
    解析json数据，并提取标题、价格、商家地址、销量、评价地址:param goods_str; string格式数据:return:
    """
    goods_json = json.loads(goods_str)
    goods_items = goods_json['mods']['itemlist']['data']['auctions']
    goods_list = []
    for goods_item in goods_items:
        goods = {'title': goods_item['raw_title'],
                 'price': goods_item['view_price'],
                 'location': goods_item['item_loc'],
                 'sales': goods_item['view_sales'],
                 'comment_url': goods_item['comment_url']}
        goods_list.append(goods)
    return goods_list

In [90]:
#  下面的cookie要改成自己的，在Chrome浏览器中按F12，然后在Network中找到请求的url，然后在Headers中找到cookie
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36", "cookie": "miid=378718641394829915; cookie2=1f955ab6ead9bd3badb3942c7ce0f987; t=ec42adcfb6f1a32a0dd7b9173e1f11d2; _samesite_flag_=true; cna=3RTEHJthAHUCAdJXyrfxrM45; cancelledSubSites=empty; dnk=zzysh; publishItemObj=Ng%3D%3D; tracknick=zzysh; _tb_token_=75b84ae861ee0; _utk=VocP@qJyn^AtWdm; alitrackid=www.taobao.com; lastalitrackid=www.taobao.com; _gid=GA1.2.737213218.1688174269; xlly_s=1; _uetsid=17a4a03017ad11eea005bbb36e61fce6; _uetvid=17a48e8017ad11ee8aefdb44d5fbf391; _ga=GA1.2.1700763778.1688174268; _ga_YFVFB9JLVB=GS1.1.1688174268.1.1.1688174277.0.0.0; sgcookie=E1002%2FOGF9%2FFi3gVil1fxmn7g2f2Q3nYai1%2BATFkYIdxxmfTLTBXm8qfBIWhMVOAgKK9XUz9GC%2FY8E5axVouTiY8QG9P%2B5%2F9%2B5Xpo2D0C%2BoSX2GM0a2J4AtAi2GlDO2Ui8WGi68kyuM%2FHyJeyz3uSIx1Uw%3D%3D; unb=10161714; uc3=vt3=F8dCsGO1TudAezwcj4c%3D&id2=UoH%2B4Nq1hrg%3D&nk2=GdImh2M%3D&lg2=VFC%2FuZ9ayeYq2g%3D%3D; csg=168deaa3; lgc=zzysh; cookie17=UoH%2B4Nq1hrg%3D; skt=8dbd453c556bfbf9; existShop=MTY4ODE3NDMxMQ%3D%3D; uc4=nk4=0%40Gxs1ktWfs6H3hz4OO8%2BwLQ%3D%3D&id4=0%40UOnhBNnhsrrsmq44E%2FqCMrDhPA%3D%3D; _cc_=V32FPkk%2Fhw%3D%3D; _l_g_=Ug%3D%3D; sg=h48; _nk_=zzysh; cookie1=VqoKHhjjCyaaWJGxSrsLW9hrRfSFdkB%2BVujxmtrYbGM%3D; _m_h5_tk=a386909c1f47fe25da1a6d40b67011e2_1688190864423; _m_h5_tk_enc=9916dd5d5f1dcfbba73ffc47e0651f10; mt=ci=1_1; uc1=pas=0&cookie14=Uoe8gqFXMmBTNw%3D%3D&cookie16=U%2BGCWk%2F74Mx5tgzv3dWpnhjPaQ%3D%3D&cookie15=W5iHLLyFOGW7aA%3D%3D&existShop=false&cookie21=WqG3DMC9EdFp7qHvqFD7pg%3D%3D; v=0; hng=CN%7Czh-CN%7CCNY%7C156; thw=cn; useNativeIM=false; tfstk=da9W3hTsJep4Xt_Bq8i2G9NcLZ6BFUMaPkspjHezvTBRJItMRaonZTRfRELfy8ooZ6QBrTCHaw7PR9tH5muZQA-kq9Xr7VkaCzBFUYmZFxlq23XhpVr4pfk6qFccSNssGcz-gyK_l-wYBaOmfClxBApfD2b9VLIc0Ks5NNCxFJXwVR7sUPNQc12CcNojcWV36Tr8d; l=fBQfiEPINHxgOOLDBOfwnurza77tGIRfguPzaNbMi9X19u5UddWAUHZJSJZ6Q3QQETfczetPl_pr6dEeyka3WxgKqelyRs5mp-p6-bpU-L5..; isg=BDAwa-x_IKw23fxbzCocMm6KAfiCeRTD-mitICqBmQsG5dGP0o11Uo2XOe2F9cyb; JSESSIONID=FBDB4E4299376A7A069341EB48B4CDEF"
}

urls = Geturls("眼镜", 5)
goods_list = []
for url in urls:
    html = GetHtml(url, headers)

    goods_match = re.search(r'g_page_config = (.*?)}};', html.text)
    goods_str = goods_match.group(1) + '}}'
    goods_info = get_goods_info(goods_str)
    goods_list.append(goods_info)

    time.sleep(5)

# 将二维list扁平化 参考 https://note.nkmk.me/en/python-list-flatten/
flatten_goods_list = sum(goods_list, [])
print(len(flatten_goods_list))
#

224


In [91]:
flatten_goods_list[0]["title"]

'德国蔡司防蓝光镜片成品可配有度数超轻纯钛近视眼镜框男大脸显瘦'

##### 保存商品数据到数据库


In [92]:
# 模拟生成商品信息数据
products = []

for goods in flatten_goods_list:
    name = goods["title"]
    price = goods["price"]
    products.append((name, price))

# 插入商品信息数据
insert_product = "INSERT INTO Product (name, price) VALUES (%s, %s)"
cursor.executemany(insert_product, products)


# 提交事务
cnx.commit()

## 生成客户数据


In [93]:
# 模拟生成客户资料数据
customers = []
fake = faker.Faker('zh_CN')
for region in regions:
    customer_count = random.randint(100, 200)  # 生成随机数量的客户
    for _ in range(customer_count):
        company_name = fake.company()
        customers.append((company_name, region))

# 插入客户资料数据
insert_customer = "INSERT INTO Customer (name, region_id) VALUES (%s, %s)"
cursor.executemany(insert_customer, customers)

# 提交事务
cnx.commit()

## 生成销售数据


In [148]:
import datetime


def daterange(start_date, end_date):
    """生成日期范围内的所有日期

    Args:
        start_date (_type_): 开始日期
        end_date (_type_): 结束日期

    Yields:
        _type_: date
    """
    for n in range((end_date - start_date).days + 1):
        yield start_date + datetime.timedelta(n)


# 模拟生成销售数据
start_date = datetime.date(2023, 1, 1)
end_date = datetime.date(2023, 6, 30)
sales_data = []

for region in regions:
    # 查询当前大区的销售人员列表
    query_salesperson = f"SELECT id FROM Salesperson WHERE region_id = {region}"
    cursor.execute(query_salesperson)
    salespersons = cursor.fetchall()

    # 查询商品列表
    query_product = "SELECT id, name, price FROM Product"
    cursor.execute(query_product)
    products = cursor.fetchall()

    # 查询当前大区的客户列表
    query_customer = f"SELECT id FROM Customer WHERE region_id = {region}"
    cursor.execute(query_customer)
    customers = cursor.fetchall()

    for month in range(1, 7):  # 遍历每个月份
        # 获取当前月份的起始日期和结束日期
        month_start = datetime.date(2023, month, 1)
        if month == 6:
            month_end = datetime.date(2023, 6, 30)
        else:
            month_end = datetime.date(
                2023, month + 1, 1) - datetime.timedelta(days=1)

        # 随机生成每个月的销售记录数量
        sales_count = random.randint(50, 150)

        for _ in range(sales_count):
            # 随机选择日期
            sale_date = random.choice(
                [date for date in daterange(month_start, month_end)])
            sale_date_str = sale_date.strftime("%Y-%m-%d")

            salesperson_id = random.choice(salespersons)[0]
            product = random.choice(products)
            product_id = product[0]
            product_price = product[2]
            customer_id = random.choice(customers)[0]
            quantity = random.randint(1, 10)
            total = quantity * product_price

            sales_data.append((salesperson_id, product_id,
                              customer_id, region, quantity, total, sale_date_str))

# 插入销售数据
insert_sales_data = "INSERT INTO SalesData (salesperson_id, product_id, customer_id, region_id, amount, total, sales_date) VALUES (%s, %s, %s, %s, %s, %s, %s)"
cursor.executemany(insert_sales_data, sales_data)

# 提交事务
cnx.commit()

In [113]:
query_product = "SELECT id, name, price FROM Product"
cursor.execute(query_product)
products = cursor.fetchall()

product = random.choice(products)

print(product)

print(product[0], product[1], product[2])

(217, 'moody甜心半年抛美瞳[1副需拍2片]彩色隐形眼镜男女官方旗舰店', Decimal('69.90'))
217 moody甜心半年抛美瞳[1副需拍2片]彩色隐形眼镜男女官方旗舰店 69.90


## 关闭数据库连接


In [96]:
cursor.close()
cnx.close()